# Transformation Start, Ts Calculator

This notebook is for the analysis of dilation curves, primarily for steel alloys with allotromorphic phase transformations, for the determination of transformation start temperatures. The calculator adopts the offset method, first proposed by Yang and Bhadeshia, to predict the temperature at which 1% of a transformation has occurred. For more information regarding the offset method see H-S. Yang and H.K.D.H. Bhadeshia, Materials Science and Technology 23 (2007) 556-560. https://www.tandfonline.com/doi/abs/10.1179/174328407X176857. 

## 1. Input Parameters

The input parameters for this calculator are: **file**, **sample**, **L0**, **offset**, and **ranges**.

### Parameter 1: file

The file parameter is for the input of the pure .asc dilatometry file to be analysed. The format of the .asc file should only contain a column for temperature and a column for change in length (in that order). An example input file can be viewed along side this notebook titled '**dilation_file_example.asc**' for clarification. 

An example input of this parameter would be:

    file = 'dilation_file_example.asc'

### Parameter 2: sample

This parameter is for inputting the name of the sample being analysed. This is simply to help distinguish between multiple samples being analysed at once and will appear above the analysed data.

An example of this input, for a SA-540 steel sample held at 870°C and cooled at 1°C/s, could be as follows:

    sample = 'SA-540, TA=870C, Q=1C/s'

### Parameter 3: L0

The L0 parameter is the value for the initial length of the dilatometer sample (in metres). This value is used to calculate the sample strain from the measured change in length. 

An example of this input would be:

    L0 = 0.01

### Parameter 4: offset

This value is used to offset the linear, untransformed gradient line to predict the transformation start temperature at the intercept of the offset line. More information regarding this technique can be found in the '**Dilatometer quenching tests**' experiment document associated with these analysis codes. 

The value of this offset can be calculated using another Jupyter notebook available with these analysis codes called '**Offset_Calculator.ipynb**'. 

An example of this input would be:

    offset = 0.00011672

### Parameter 5: ranges

This parameter is for inputting the temperature ranges in which to measure the gradient of the untransformed line to be offset. It is recommended that 3 different ranges are selected so that any uncertainty can be measured. Temperature ranges should be at least 50°C away from the transformation start temperature and over at least a 50°C range. 

The input of this parameter should be as a nested list, the form of which is:

    nested_list = [[T11,T21],[T12,T22],[T13,T23]]
    
An example of the inputted temperature ranges for a Ts ~ 450°C could be:

    ranges = [[500,550],[550,600],[600,650]]

## 2. Example

See '**transformation_analysis_example.png**' for an example of how to use this calculator.

## 3. Code

In [1]:
# Import required packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
        
def Ts_Calculator(file,sample,L0,offset,ranges):
    
    # Read .asc file and calculate strain
    data = pd.read_csv('test.asc', delimiter = r"\s+", header = None, skiprows=4, usecols = range(1,3), error_bad_lines=False, engine='python')
    data.columns = ['T', 'x']
    data['e'] = (data['x']*10**(-6))/L0
    
    # Isolate the cooling curve (Note-to-self: this may need altering for complex treatments)
    dataset = data.tail(n = int(len(data)/2))
    Temp = np.array(dataset['T'])
    strain = np.array(dataset['e'])
    
    # Plot dilation curve 
    data.plot(x = 'T', y = 'e', figsize = (16,6), color = 'seagreen', label = 'Dilation Curve')
    plt.title(sample+' Dilation Analysis', fontsize = 16)
    plt.xlabel('Temperature (\N{DEGREE SIGN}C)', fontsize = 12)
    plt.ylabel('Strain (m/m)', fontsize = 12)
    plt.show()
    
    # Calculate gradient/s and isolate intercept with offsetted line
    intercepts = []
    m = 0
    fig, axs = plt.subplots(1, len(ranges), figsize = (16,5))
    for x in ranges: 
        i = []
        y = []
        y_off = []
        for n in range(len(dataset)):
            if round(dataset.iloc[n].loc['T'],0) == x[0]:
                e0 = dataset.iloc[n].loc['e']
            if round(dataset.iloc[n].loc['T'],0) == x[1]:
                e1 = dataset.iloc[n].loc['e']

        g = (e1-e0)/(x[1]-x[0])
        c = e0-(x[0]*g)+offset

        for T in Temp:
            y_off.append(T*g+c)
            y.append(T*g+c-offset)
          
        idx = np.argwhere(np.gradient(np.sign(np.array(y_off) - np.array(strain)))).flatten()   
        for n in range(len(idx)):
            if Temp[idx[n]] < x[0] and Temp[idx[n]] > (x[0]-250):
                i.append(round(Temp[idx[n]],1))
        try:      
            intercept = i[0]
            intercepts.append(intercept)
        except IndexError:
            return print('No intercept found for range: [',x[0],',',x[1],']')
        
        # Plot graphs showing interception points
        if len(ranges) > 1:
            axs[m].plot(dataset['T'], dataset['e'], label = 'Dilation Curve')
            axs[m].set_xlim(x[1]-300,x[1])
            axs[m].set_ylim(intercept*g+c-0.0005,x[1]*g+c+0.0005)
            axs[m].plot(Temp, y, linestyle = '-', color = 'k', linewidth = 1)
            axs[m].plot(Temp, y_off, linestyle = '--', color = 'k', linewidth = 1, label = 'Offset Fit')
            axs[m].scatter(intercept,intercept*g+c, marker = 'o', color = 'r')
            axs[m].text(intercept+25, intercept*g+c, str(intercept), fontsize = 12, bbox={'facecolor': 'white'}, color = 'red')

            axs[m].set_title('Slope Range: '+str(x[0])+' - '+str(x[1])+'\N{DEGREE SIGN}C', fontsize = 14)
            axs[m].set_xlabel('Temperature (\N{DEGREE SIGN}C)', fontsize = 12)
            if m == 0:
                axs[m].set_ylabel('Strain (m/m)', fontsize = 12)
            else:
                pass
            m = m + 1
        else:
            plt.plot(dataset['T'], dataset['e'], label = 'Dilation Curve')
            plt.xlim(x[1]-300,x[1])
            plt.ylim(intercept*g+c-0.0005,x[1]*g+c+0.0005)
            plt.plot(Temp, y, linestyle = '-', color = 'k', linewidth = 1)
            plt.plot(Temp, y_off, linestyle = '--', color = 'k', linewidth = 1, label = 'Offset Fit')
            plt.scatter(intercept,intercept*g+c, marker = 'o', color = 'r')
            plt.text(intercept+25, intercept*g+c, str(intercept), fontsize = 12, bbox={'facecolor': 'white'}, color = 'red')

            plt.title('Slope Range: '+str(x[0])+' - '+str(x[1])+'\N{DEGREE SIGN}C', fontsize = 14)
            plt.xlabel('Temperature (\N{DEGREE SIGN}C)', fontsize = 12)
            plt.ylabel('Strain (m/m)', fontsize = 12)
    
    print('Start Temperature = ', str(round(np.mean(intercepts),1)), u'\xb1',str(round(np.std(intercepts),2)), '\N{DEGREE SIGN}C')
    plt.show()
    return